In [1]:
#数据集来源kaggle:HR Analytics
import torch
import numpy as np
import pandas as pd

In [ ]:
#1.查看数据
url = "./HR-Employee-Attrition.csv" 

In [ ]:
data = pd.read_csv(url)

In [ ]:
data.head()

In [ ]:
data.shape
data.info()

In [ ]:
data.columns

In [ ]:
#2.预处理数据集，one-hot处理

In [ ]:
BusinessTravel_0 = pd.get_dummies(data.BusinessTravel)
Department_0 = pd.get_dummies(data.Department)
EducationField_0 = pd.get_dummies(data.EducationField)
Gender_0 = pd.get_dummies(data.Gender)
JobRole_0 = pd.get_dummies(data.JobRole)
MaritalStatus_0 = pd.get_dummies(data.MaritalStatus)
Over18_0 = pd.get_dummies(data.Over18)
OverTime_0 = pd.get_dummies(data.OverTime)


#删除多列
data = data.drop(['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime'],axis=1)

In [ ]:
list = [BusinessTravel_0,Department_0,EducationField_0,Gender_0,JobRole_0,MaritalStatus_0,Over18_0,OverTime_0]
for item in list:
    data = data.join(item,rsuffix='_r')


In [ ]:
data

In [ ]:
#处理特征数据YES-1 NO-0
data['Attrition'] = data['Attrition'].replace(["Yes","No"],[1,0])

In [ ]:
data

In [ ]:
#3.划分训练集和测试集
all_counts = len(data)
train_counts = int(all_counts*0.7)
test_counts = all_counts - train_counts

train_data  = data.iloc[:train_counts,:]
test_data = data.iloc[train_counts:,:]

In [ ]:
#4.配置输入数据和输出数据 X,Y
Y = torch.from_numpy(train_data.iloc[:,1:2].values).type(torch.float32)

In [ ]:
Y

In [ ]:
del train_data['Attrition']
X = torch.from_numpy(train_data.values).type(torch.float32)

In [ ]:
train_data

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
#5.创建多层感知机
from torch import nn 

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(55,64)
        self.linear_2 = nn.Linear(64,64)
        self.linear_3 = nn.Linear(64,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self,input):
        x = self.linear_1(input)
        x = self.relu(x)
        x = self.linear_2(x)
        x = self.relu(x)
        x = self.linear_3(x)
        x = self.sigmoid(x)
        return x

In [ ]:
model = Model()
#设置学习率和优化函数
lr = 0.0001
opt = torch.optim.Adam(model.parameters(),lr=lr)

In [ ]:
model.parameters()

In [ ]:
#6.训练
#损失函数
loss_fun = nn.BCELoss()
#batch
batch = 64
n_batches = len(train_data)//batch
#epoch
epochs = 5000

In [ ]:
for epoch in range(epochs):
    for i in range(n_batches):
        start = i*batch
        end = start + batch
        x = X[start:end]
        y = Y[start:end]
        y_pred = model(x)
        loss = loss_fun(y_pred,y)
        #反向传播前先将梯度置为0
        opt.zero_grad()
        #反向传播
        loss.backward()
        opt.step()
        

In [ ]:
loss_fun(model(X),Y)

In [ ]:
data['Attrition'].value_counts()

In [ ]:
27/(1233+237)

In [ ]:
((model(X).data.numpy()>0.5).astype('int') == Y.numpy()).mean()